# Commonly Used Functions

### Missing Values

In [ ]:
# Drop all rows that have missing values
data.dropna(axis=0, how='any')

In [ ]:
# Replace missing values 

data['LotFrontage'].fillna(0, inplace=True) # replace missing values with 0
data['MasVnrType'].fillna("None", inplace=True)  # Fill missing values for MasVnrType with "none"

In [ ]:
# Count up total missing values for each var
data.isnull().sum()

# Find total number of missing values in data
data.isnull().sum().sum()

In [ ]:
# For rows where "Electrical" variable is null, are there any other var's that contain null values?
np.unique(data[data['Electrical'].isnull()==True].isnull().any(axis=0).tolist(), return_counts=True)

### Replace Values

In [ ]:
# Correct data quality issue: Replace MasVnrArea with 0 when MasVnrType is None
data.loc[(data['MasVnrType'] == "None") & (data['MasVnrArea']!=0), 'MasVnrArea'] = 0

In [ ]:
# Make new variables
data['HasBsmt'] = 1
data.loc[data['BsmtCond'].isnull(), 'HasBsmt'] = 0

In [ ]:
# Conditional Replacing
data.loc[data['latitude']==-2e-08, 'latitude'] = np.mean(data[data['latitude']!=-2e-08]['latitude'])

In [ ]:
# Using replace function
data['CentralAir'] = data['CentralAir'].replace(['N', 'Y'], [0, 1])

### Conditionally Drop Data

In [ ]:
# Drop rows where BsmtFinType2 is "unfinished" and where BsmtExposure is "null"
data = data.drop(data.index[np.where((data['BsmtFinType2']=='Unf') & (data['BsmtExposure'].isnull()))])

In [ ]:
# Drop rows where BsmtUnfSF is greater than 0 and either BsmtExposure is null or BsmtFinType2 is null
data = data.drop(data.index[np.where((data['BsmtUnfSF']>0) & (data['BsmtExposure'].isnull() | data['BsmtFinType2'].isnull()))])

### Examine Data

In [ ]:
# Count Values

# what are the different housing types when Masonry Type is none
data[data['MasVnrType']=='None']['House Type'].value_counts(dropna=False)

In [ ]:
# Look at data variables when the condition for another variables is true
data[data['BsmtExposure'].isnull()==True][['BsmtQual', 'BsmtCond', 'BsmtFinSF1']]

In [ ]:
# Find number of unique values in a column
len(data['funder'].value_counts().index.unique()) # Find number of funders

# View sorted values
sorted(data['latitude'].value_counts().index.unique())

### Date Times

In [ ]:
# Change value to date time
data['date_recorded'] = pd.to_datetime(data['date_recorded'])

# Extract month / year from date time
data['month-year'] = data['date_recorded'].map(lambda x: str(x.year) +"-"+str(x.month))



### Data Exploration - Graphically

In [ ]:
# Find correlations
corr_all = data.corr(method='pearson')
corrs2 = corr_all.ix[-1][:-1]
corrs2_dict = corrs2.to_dict()
corrs2_dict

# Reorder by level of correlation
print("FEATURE \tCORRELATION")
for attr in sorted(corrs2_dict.items(), key = lambda x: -abs(x[1])):
    print("{0}: \t{1}".format(*attr))

In [ ]:
# Make histogram of all data variables within the data
data.hist(figsize=(16, 20), xlabelsize = 8, ylabelsize = 9)

### Variable Encoding

In [ ]:
# Ordered Encoding

# Encode Exterior Quality with the following as the quality order
overallqual = ['Po', 'Fa', 'TA', 'Gd', 'Ex']
data.ExterQual= data.ExterQual.astype("category", ordered=True, categories=overallqual).cat.codes

In [ ]:
# One Hot Encoding
varsdf = pd.get_dummies(varsdf, columns=['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 
                                            'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
                                            'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
                                            'Heating', 'Functional', 'GarageType', 
                                            'GarageFinish', 'PavedDrive', 'SaleType', 'Electrical', 'SaleCondition'])

### Merge data

In [ ]:
data_org = pd.merge(indep_vars, outcome, on='id')

### Normalization

In [ ]:
# Take log to normalize variable
data['SalePrice'] = np.log1p(data['SalePrice'])

In [ ]:
# Look at skew
stats.skew(data[i])

### Scaling

In [ ]:
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(scale_df1)
scaled_df = pd.DataFrame(scaled_df, columns = cont_vars)

In [ ]:
# Concatenate Data - combine columns
stand_df = pd.concat([scaled_df, varsdf], axis=1)

In [ ]:
# Using robust scaling
scaler = preprocessing.RobustScaler()

varsdf = pd.DataFrame(data[['amount_tsh', 'gps_height']])

z = scaler.fit_transform(varsdf)

amount_tsh_rob = []
gps_height = []
for x in range(len(z)):
    amount_tsh_rob.append(z[x][0])
    gps_height.append(z[x][1])

amount_tsh_rob = pd.Series(amount_tsh_rob)
gps_height = pd.Series(gps_height)

plt.scatter(amount_tsh_rob, gps_height)
plt.scatter(data['amount_tsh'], data['gps_height'])

# first store original index
index_orig = data.index # type index
datacopy = data.copy()
# Then reindex
data = data.reset_index(drop=True)

data['amount_tsh_rob'] = amount_tsh_rob
data['gps_height_rob'] = gps_height

### Regex

In [ ]:
import re

religious = ['[M,m].*si*l[ie][mus]+', '[C,c]hur[ch]+', '[S,s]ain[ts]', '[C,c].*rist[ia]+[ns]+', '[D,d]ioce[se]*', 
             '[Aa]nglican', '[Mm]ethodist', '[Mm]i+s+i+on+ar', '[Is]lam', 'Neemia Mission', '^Rc.*', '[C,c]ath.*ic', 
             '^[R,r]oman.*', 'Isla$', '[Bb].ptist', '^Missio$']

# If the string matches a logic in the above list, then replace funder type var with value "Private Company"
if re.search("|".join(relgious), i):
        data.loc[data['funder']==i, 'funder_type'] = "Private Company"

### Write out data

In [ ]:
data_trans.to_csv("Ames_Normalized.csv", index=False)